Ref: https://learn.microsoft.com/en-us/dotnet/machine-learning/how-to-guides/save-load-machine-learning-models-ml-net

## 0. Add & Import packages

In [13]:
#r "nuget: Microsoft.ML"
#r "nuget: Microsoft.ML.DataView"

Installed Packages Microsoft.ML, 4.0.2 microsoft.ml.dataview, 4.0.2

In [14]:
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms.Text;
using Microsoft.ML.Trainers;

## 1. Define data models
### a. Input data

In [15]:
public class HousingData
{
    [LoadColumn(0)]
    public float Size { get; set; }

    [LoadColumn(1, 3)]
    [VectorType(3)]
    public float[] HistoricalPrices { get; set; }

    [LoadColumn(4)]
    [ColumnName("Label")]
    public float CurrentPrice { get; set; }
}

### b. Output data
Like the Features and Label input column names, ML.NET has default names for the predicted value columns produced by a model. The names can differ depending on the task.
Because the algorithm used in this sample is a linear regression algorithm, the default name of the output column is Score, which is defined by the ColumnName attribute on the PredictedPrice property.

In [16]:
class HousingPrediction
{
    [ColumnName("Score")]
    public float PredictedPrice { get; set; }
}

## 2. Set up data, pipeline and training

In [ ]:
HousingData[] housingData = new HousingData[]
{
    new HousingData
    {
        Size = 600f,
        HistoricalPrices = new float[] { 100000f, 125000f, 122000f },
        CurrentPrice = 170000f
    },
    new HousingData
    {
        Size = 1000f,
        HistoricalPrices = new float[] { 200000f, 250000f, 230000f },
        CurrentPrice = 225000f
    },
    new HousingData
    {
        Size = 1000f,
        HistoricalPrices = new float[] { 126000f, 130000f, 200000f },
        CurrentPrice = 195000f
    }
};

// Create MLContext
MLContext mlContext = new MLContext();

// Load Data
IDataView data = mlContext.Data.LoadFromEnumerable<HousingData>(housingData);

// Define data preparation estimator
EstimatorChain<RegressionPredictionTransformer<LinearRegressionModelParameters>> pipelineEstimator =
    mlContext.Transforms.Concatenate("Features", new string[] { "Size", "HistoricalPrices" })
        .Append(mlContext.Transforms.NormalizeMinMax("Features"))
        .Append(mlContext.Regression.Trainers.Sdca());


## 3. Save model
### a. Save a model locally
When saving a model, you need two things:
1. The ITransformer of the model.
2. The DataViewSchema of the ITransformer's expected input.
- After training the model, use the Save method to save the trained model to a file called model.zip using the DataViewSchema of the input data.

In [ ]:
// Save Trained Model
mlContext.Model.Save(trainedModel, data.Schema, "./ExportedModels/model.zip");

### b. Save an ONNX model locally
To save an Open Neural Network Exchange (ONNX) version of your model locally, you must install the Microsoft.ML.OnnxConverter NuGet package.

With the OnnxConverter package installed, you can use it to save your model into the ONNX format. This requires a Stream object, which you can provide as a FileStream using the File.Create method. The File.Create method takes a string parameter that represents the path of the ONNX model.

In [26]:
#r "nuget: Microsoft.ML.OnnxConverter"
#r "nuget: Microsoft.ML.OnnxTransformer"

Installed Packages Microsoft.ML.OnnxConverter, 0.22.2 Microsoft.ML.OnnxTransformer, 4.0.2

In [27]:
using (var stream = File.Create("./ExportedModels/onnx_model.onnx"))
{
    // With the stream created, you can call the ConvertToOnnx method and give it the trained model, 
    // the data used to train the model, and the stream. However, not all trainers and transformers
    // are exportable to ONNX. For a complete list, see the Transforms and How to Choose an ML.NET Algorithm guides
    mlContext.Model.ConvertToOnnx(trainedModel, data, stream);
}

## 3. Load a model
### a. Load a model stored locally
Models stored locally can be used in other processes or applications like ASP.NET Core and serverless web apps. For more information, see Use ML.NET in Web API and Deploy ML.NET Serverless Web App.

In a separate application or process, use the Load method along with the file path to get the trained model into your application.

In [28]:
//Define DataViewSchema for data preparation pipeline and trained model
DataViewSchema modelSchema;

// Load trained model
ITransformer trainedModel = mlContext.Model.Load("./ExportedModels/model.zip", out modelSchema);

### b. Load an ONNX model locally
To load in an ONNX model for predictions, install the Microsoft.ML.OnnxTransformer NuGet package.

With the OnnxTransformer package installed, you can load an existing ONNX model by using the ApplyOnnxModel method. The required parameter is a string that's the path of the local ONNX model.

In [30]:
using Microsoft.ML.Transforms.Onnx;

OnnxScoringEstimator estimator = mlContext.Transforms.ApplyOnnxModel("./ExportedModels/onnx_model.onnx");

The ApplyOnnxModel method returns an OnnxScoringEstimator object. First, you need to load in the new data

In [31]:
HousingData[] newHousingData = new HousingData[]
{
    new()
    {
        Size = 1000f,
        HistoricalPrices = new[] { 300_000f, 350_000f, 450_000f },
        CurrentPrice = 550_00f
    }
};

Load the new data into an IDataView using the LoadFromEnumerable method.

In [32]:
IDataView newHousingDataView = mlContext.Data.LoadFromEnumerable(newHousingData);

Now you can use the new IDataView to fit on the new data.

In [33]:
estimator.Fit(newHousingDataView);

After using the Fit method on an estimator from ApplyOnnxModel, it can then be saved as a new model using the Save method described in the save a model locally section.

### c. Load a model stored remotely
To load data preparation pipelines and models stored in a remote location into your application, use a Stream instead of a file path in the Load method.

In [ ]:
// Create MLContext
MLContext mlContext = new MLContext();

// Define DataViewSchema and ITransformers
DataViewSchema modelSchema;
ITransformer trainedModel;

// Load data prep pipeline and trained model
using (HttpClient client = new HttpClient())
{
    Stream modelFile = await client.GetStreamAsync("<YOUR-REMOTE-FILE-LOCATION>");

    trainedModel = mlContext.Model.Load(modelFile, out modelSchema);
}

## 4. Work with separate data preparation and model pipelines
Working with separate data preparation and model training pipelines is optional. Separation of pipelines makes it easier to inspect the learned model parameters. For predictions, it's easier to save and load a single pipeline that includes the data preparation and model training operations.
When working with separate data preparation pipelines and models, the same process as single pipelines applies. The difference is that both pipelines need to be saved and loaded simultaneously.

Given separate data preparation and model training pipelines:

In [34]:
// Define data preparation estimator
IEstimator<ITransformer> dataPrepEstimator =
    mlContext.Transforms.Concatenate("Features", new string[] { "Size", "HistoricalPrices" })
        .Append(mlContext.Transforms.NormalizeMinMax("Features"));

// Create data preparation transformer
ITransformer dataPrepTransformer = dataPrepEstimator.Fit(data);

// Define StochasticDualCoordinateAscent regression algorithm estimator
var sdcaEstimator = mlContext.Regression.Trainers.Sdca();

// Pre-process data using data prep operations
IDataView transformedData = dataPrepTransformer.Transform(data);

// Train regression model
RegressionPredictionTransformer<LinearRegressionModelParameters> trainedModel = sdcaEstimator.Fit(transformedData);


### a. Save data preparation pipeline and trained model
To save both the data preparation pipeline and trained model, use the following commands:

In [35]:
// Save Data Prep transformer
mlContext.Model.Save(dataPrepTransformer, data.Schema, "data_preparation_pipeline.zip");

// Save Trained Model
mlContext.Model.Save(trainedModel, transformedData.Schema, "model.zip");

### b. Load data preparation pipeline and trained model

In [ ]:
// Create MLContext
MLContext mlContext = new MLContext();

// Define data preparation and trained model schemas
DataViewSchema dataPrepPipelineSchema, modelSchema;

// Load data preparation pipeline and trained model
ITransformer dataPrepPipeline = mlContext.Model.Load("data_preparation_pipeline.zip",out dataPrepPipelineSchema);
ITransformer trainedModel = mlContext.Model.Load("model.zip", out modelSchema);